In [1]:
import shapefile, os

buildings_sf = shapefile.Reader( os.path.join( os.getcwd(), '..', 'data', 'shapefiles', 'buildings' ) )

In [2]:
import pandas as pd

buildings_df = pd.read_pickle( os.path.join( os.getcwd(), '..', 'data', 'profiles', 'gas_consumption.pkl' ) )

In [3]:
from dblayer import *
from dblayer.func.func_citydb_view_nrg import *
from dblayer.func.func_citydb_pkg import *
from dblayer.func.func_postgis_geom import *

In [4]:
connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'citydb'
    )

In [5]:
db_access = DBAccess()
db_access.connect_to_citydb( connect )

In [6]:
db_access.cleanup_citydb_schema()
db_access.cleanup_simpkg_schema()

In [7]:
srid=4326

In [8]:
import datetime

for data in buildings_sf:
    building_name = data.record['name']
    building_energy_carrier = data.record['e_carrier']

    geom_2d_points = [ Point2D( p[0], p[1] ) for p in data.shape.points ]

    geom = db_access.execute_function( 
        geom_from_2dpolygon( geom_2d_points, srid )
        )

    geom_id = db_access.add_citydb_object(
        insert_surface_geometry,
        geometry = geom
        )

    building_id = db_access.add_citydb_object(
        insert_building,
        name = building_name,
        lod0_footprint_id = geom_id,
        )

    if building_energy_carrier == 'gas':

        gas_consumption_profile = buildings_df[building_name].tolist()

        time_series_id = db_access.add_citydb_object(
            insert_regular_time_series,
            name = 'ts_gas_consumption_{}'.format( building_name ),
            description = 'gas consumption profile of {}'.format( building_name ),
            acquisition_method = 'Simulation',
            interpolation_type = 'AverageInSucceedingInterval',
            values_array = gas_consumption_profile,
            values_unit = 'kW',
            temporal_extent_begin = datetime.datetime( 2019, 1, 1, 0, 0, 0 ),
            temporal_extent_end = datetime.datetime( 2019, 1, 1, 23, 0, 0 ),
            time_interval = 1,
            time_interval_unit = 'h'
            )

        final_energy_id = db_access.add_citydb_object(
            insert_final_energy,
            name = 'final_energy_{}'.format( building_name ),
            description = 'gas consumption of {}'.format( building_name ),
            nrg_car_type = 'NaturalGas',
            time_series_id = time_series_id
            )

        boiler_id = db_access.add_citydb_object(
            insert_boiler,
            name = 'gas_boiler_{}'.format( building_name ),
            description = 'gas boiler of {}'.format( building_name ),
            inst_in_ctyobj_id = building_id
            )

        db_access.add_citydb_object(
            insert_conv_sys_to_final_nrg,
            conv_system_id = boiler_id,
            final_nrg_id = final_energy_id,
            role = 'consumption'
            )

In [9]:
db_access.commit_citydb_session()